In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt

In [2]:
datatrx = pd.read_csv('../input/uas-rfm-wildan/UAS_RFM.csv', sep=';', parse_dates=['TrxDate'])
datatrx.head()

,CardID,TrxDate,Amount
0,1000152.0,2021-02-28,"395.863,00"
1,1000158.0,2021-05-13,"903.929,00"
2,1000156.0,2021-05-25,"288.924,00"
3,1000115.0,2021-04-02,"813.592,00"
4,1000164.0,2021-09-06,"741.387,00"


In [3]:
datatrx.head(15)

,CardID,TrxDate,Amount
0,1000152.0,2021-02-28,"395.863,00"
1,1000158.0,2021-05-13,"903.929,00"
2,1000156.0,2021-05-25,"288.924,00"
3,1000115.0,2021-04-02,"813.592,00"
4,1000164.0,2021-09-06,"741.387,00"
5,1000136.0,2021-03-16,"293.544,00"
6,1000147.0,2021-03-17,"336.628,00"
7,1000118.0,2021-04-16,"431.299,00"
8,1000167.0,2021-02-22,"788.710,00"
9,1000160.0,2021-05-14,"902.354,00"


In [4]:
datatrx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   CardID   199 non-null    float64       
 1   TrxDate  199 non-null    datetime64[ns]
 2   Amount   199 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 23.5+ KB


In [5]:
print(datatrx['TrxDate'].min(), datatrx['TrxDate'].max())

2021-01-01 00:00:00 2021-12-06 00:00:00


In [6]:
sd = dt.datetime(2021,12,19)
datatrx['hist']=sd - datatrx['TrxDate']
datatrx['hist'].astype('timedelta64[D]')
datatrx['hist']=datatrx['hist'] / np.timedelta64(1, 'D')
datatrx.head(15)

,CardID,TrxDate,Amount,hist
0,1000152.0,2021-02-28,"395.863,00",294.0
1,1000158.0,2021-05-13,"903.929,00",220.0
2,1000156.0,2021-05-25,"288.924,00",208.0
3,1000115.0,2021-04-02,"813.592,00",261.0
4,1000164.0,2021-09-06,"741.387,00",104.0
5,1000136.0,2021-03-16,"293.544,00",278.0
6,1000147.0,2021-03-17,"336.628,00",277.0
7,1000118.0,2021-04-16,"431.299,00",247.0
8,1000167.0,2021-02-22,"788.710,00",300.0
9,1000160.0,2021-05-14,"902.354,00",219.0


In [7]:
datatrx=datatrx[datatrx['hist'] < 730]
datatrx.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 199 entries, 0 to 198
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   CardID   199 non-null    float64       
 1   TrxDate  199 non-null    datetime64[ns]
 2   Amount   199 non-null    object        
 3   hist     199 non-null    float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 7.8+ KB


In [8]:
rfmTable = datatrx.groupby('CardID').agg({'hist': lambda x:x.min(), # Recency
                                        'CardID': lambda x: len(x), # Frequency
                                        'Amount': lambda x: x.sum()}) # Monetary Value

rfmTable.rename(columns={'hist': 'recency', 
                         'CardID': 'frequency', 
                         'Amount': 'monetary_value'}, inplace=True)
rfmTable.head(15)

,recency,frequency,monetary_value
CardID,,,
1000101.0,249.0,1.0,"831.579,00"
1000102.0,188.0,3.0,"734.550,00955.891,00416.789,00"
1000103.0,46.0,1.0,"635.895,00"
1000104.0,271.0,2.0,"141.151,00311.647,00"
1000105.0,13.0,5.0,"63.678,00354.097,00932.962,00490.377,00145.029,00"
1000106.0,233.0,2.0,"101.465,00114.824,00"
1000107.0,108.0,3.0,"292.238,00369.012,00445.940,00"
1000108.0,176.0,3.0,"189.070,00525.760,00880.122,00"
1000109.0,280.0,2.0,"280.728,00717.973,00"


In [9]:
quartiles = rfmTable.quantile(q=[0.20,0.40,0.60])
print(quartiles, type(quartiles))

     recency  frequency
0.2    104.0        1.0
0.4    171.0        2.0
0.6    188.0        3.0 <class 'pandas.core.frame.DataFrame'>


In [10]:
def RClass(x,p,d):
    if x <= d[p][0.20]:
        return 4
    elif x <= d[p][0.40]:
        return 3
    elif x <= d[p][0.60]: 
        return 2
    else:
        return 1
    
def FMClass(x,p,d):
    if x <= d[p][0.20]:
        return 1
    elif x <= d[p][0.40]:
        return 2
    elif x <= d[p][0.60]: 
        return 3
    else:
        return 4

In [13]:
rfmSeg = rfmTable
rfmSeg['R_Quartile'] = rfmSeg['recency'].apply(RClass, args=('recency',quartiles,))
rfmSeg['F_Quartile'] = rfmSeg['frequency'].apply(FMClass, args=('frequency',quartiles,))

In [14]:
rfmSeg['RFMClass'] = rfmSeg.R_Quartile.map(str) \
                            + rfmSeg.F_Quartile.map(str) \

rfmSeg.head(15)

,recency,frequency,monetary_value,R_Quartile,F_Quartile,RFMClass
CardID,,,,,,
1000101.0,249.0,1.0,"831.579,00",1,1,11
1000102.0,188.0,3.0,"734.550,00955.891,00416.789,00",2,3,23
1000103.0,46.0,1.0,"635.895,00",4,1,41
1000104.0,271.0,2.0,"141.151,00311.647,00",1,2,12
1000105.0,13.0,5.0,"63.678,00354.097,00932.962,00490.377,00145.029,00",4,4,44
1000106.0,233.0,2.0,"101.465,00114.824,00",1,2,12
1000107.0,108.0,3.0,"292.238,00369.012,00445.940,00",3,3,33
1000108.0,176.0,3.0,"189.070,00525.760,00880.122,00",2,3,23
1000109.0,280.0,2.0,"280.728,00717.973,00",1,2,12


In [21]:
print("Pelanggan Tetap: ",len(rfmSeg[rfmSeg['RFMClass']=='435']))
print('Berlangganan: ',len(rfmSeg[rfmSeg['F_Quartile']==3]))

Pelanggan Tetap:  0
Berlangganan:  20
